## Runway database update
This notebook guides you through the process of adding or updating runways in the database of airports.

In [1]:
import json
import numpy as np
from src.ges.ges_dataset import add_or_update_runways

c:\Users\vincent.mussot\Miniconda3\envs\LARD_env\lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


### Open and visualize the database
The database is stored in the `data/` folder, but you can use any json file if needed. Note that this database `.json` file is reused during the labeling process and must remain consistent between the generation of a scenario and the labeling of the resulting images.

In [2]:
data_file = 'data/runways_database.json'
ICAO_airport_list = []
with open(data_file, 'r') as f:
    runways_database = json.load(f)
ICAO_airport_list = [airport for airport in runways_database]

print(ICAO_airport_list)

['ZBAA', 'LTAI', 'LICJ', 'LPPT', 'LIRN', 'EDDV', 'LSZH', 'LEMD', 'LWSK', 'VHHH', 'CYUL', 'VRMM', 'LFSB', 'LCPH', 'GCRR', 'EHAM', 'SAEZ', 'LFRN', 'LFRS', 'LFPO', 'LFQQ', 'LFST', 'LFMP', 'DAAG', 'VABB', 'BIRK', 'YBBN', 'CYVR', 'CYYZ', 'KIAH', 'KJFK', 'KMIA', 'KSFO', 'MDSD', 'RJAA', 'RJTT', 'WSSS', 'FMEP', 'VQPR', 'DIAP', 'KMSY', 'SRLI', 'VDPP', 'RPMD', 'VOTV', 'YMLT', 'OMAD', 'HTDA', 'FTTJ', 'LOWL', 'LGSM', 'DAAS', 'SEQM', 'RANG_2', 'RANG', 'FAKE', 'ESOK', 'ESSV', 'ESNS', 'VEBS', 'RPVP']


In [10]:
# You can directly visualize the airport database in the .json file, 
# or you can explore the runways of a specific airport as follows
data_file = 'data/runways_database.json'
airport = 'VEBS' # Replace with the chosen airport ICAO code

with open(data_file, 'r') as f:
    runways_database = json.load(f)
    for runway in runways_database[airport]:
        print("Runway", runway)
        for point in runways_database[airport][runway]:
            print("| Corner ", point)
            print("| |  latitude: ", runways_database[airport][runway][point]['coordinate']['latitude'])
            print("| | longitude: ", runways_database[airport][runway][point]['coordinate']['longitude'])
            print("| |  altitude: ", runways_database[airport][runway][point]['coordinate']['altitude'])

Runway 32
| Corner  A
| |  latitude:  20.260783701563856
| | longitude:  85.80492260057203
| |  altitude:  53.0
| Corner  B
| |  latitude:  20.26099946989214
| | longitude:  85.80526793496004
| |  altitude:  53.0
| Corner  C
| |  latitude:  20.24077472166159
| | longitude:  85.82041184792992
| |  altitude:  61.0
| Corner  D
| |  latitude:  20.240994319805523
| | longitude:  85.82074343177473
| |  altitude:  61.0
Runway 14
| Corner  A
| |  latitude:  20.24077472166159
| | longitude:  85.82041184792992
| |  altitude:  61.0
| Corner  B
| |  latitude:  20.240994319805523
| | longitude:  85.82074343177473
| |  altitude:  61.0
| Corner  C
| |  latitude:  20.260783701563856
| | longitude:  85.80492260057203
| |  altitude:  53.0
| Corner  D
| |  latitude:  20.26099946989214
| | longitude:  85.80526793496004
| |  altitude:  53.0


# Updating the database
These examples guides you through adding new runways or updating existing ones in the runways database.
- ❗️ By default, a runway corresponds to 2 runway ids, one for each side of the runway. Therefore if 2 `runway_ids` are provided, the 4 coordinates will be stored twice in the database, once for each side of the runway with the proper order. Otherwise, the 4 coordinates will be stored only once for the specific `runway_id` specified.
- ❗️ The order of the points is important: The first 2 points **MUST** correspond to the first runway indicated in the `runway_ids`. The other 2 points **must always be present**, and will correspond to the second runway ID if present. 
- ❗️ Runway IDs are strings and may contain letters. It also implies that the runway `01` and the runway `1` will be stored separately in the json file.   

In [3]:
# Common usage: adding a runway to the database
data_file = 'data/runways_database.json'
# ESOK ESSV ESNS VEBS RPVP
airport_name = '' # Existing or new airport ICAO code

runway_ids = ['', ''] # Supports 1 or 2 runway ID

# Coordinates must be a numpy array of 4 coordinates in WGS84 format:
# ==> Latitude, Longitude, Altitude in meters.
coordinates = np.array([
    [               ,            ], # Points 1 and 2 should correspond to the first runway ID
    [               ,            ],
    [               ,            ], # Points 3 and 4 should correspond to the second runway ID if present,
    [               ,            ]  # but these last two points are always required nonetheless
    ])

# This function will add the existing runway to the database or update its coordinates if it exists
add_or_update_runways(data_file, airport_name, runway_ids, coordinates)

## Advanced usage
- Using another database file (❗️ the file must exist already)

In [11]:
# You can specify another file if needed
runways_db_name = 'data/runways_database.json' # (file must exist!)
airport_name = 'YBBN'
runways = ['01', '19'] # Supports 1 or 2 runway ID.
# Pay attention to the '01' here, as it might be redundant with an existing runway '1'
coordinates = np.array([
    [-27.402915287771446, 153.11842342220433, -1 ], # Points 1 and 2 should correspond to the first runway ID
    [-27.402734367704532, 153.11802790132325, -1 ],
    [-27.374768439323887, 153.1345185366854,  -3 ], # Points 3 and 4 should correspond to the second runway ID if present
    [-27.37459000204882,  153.13412570163402, -3 ]])

add_or_update_runways(runways_db_name, airport_name, runways, coordinates)

- Building a database of buildings or any element visible in aerial images (❗️ file must exist)

In [5]:
# We could even build another type of database, for instance specifying the coordinates of buildings
runways_db_name = 'data/runways_database.json' # (file must exist!)
building_code = 'RANG_2'
building = ['RH'] # You can specify a single element, still with its 4 coordinates
coordinates = np.array([[43.61339381397081, 1.4000701997991523,160],
                        [43.61324525478013, 1.4002130274256368, 160],
                        [43.613138447292116, 1.40001454396348,  160],
                        [43.61328603577096, 1.3998697046802842, 160]
                        ])

add_or_update_runways(runways_db_name, building_code, building, coordinates)